In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
from models.dnn import DNN
from preprocessing.encoding import seqToTensor

%load_ext autoreload
%autoreload 2

In [3]:
from torch.utils.data import DataLoader
from preprocessing.csv_dataset import CSVDataset

# Load Data

In [4]:
# Just a quick glimpse at the data

In [5]:
data = pd.read_csv("data/preprocessed-1-dnn.csv")
data

,TrainSeq,LastNum
0,"1,1,9,1,25,11,1,49,31,43,1,81,61,247,85,1,121,...",000657871
1,"1,0,0,1,0,0,1,0,0,1,0,0,2,0,0,2,0,0,2,0,0,2,0,...",000000000
2,"1,1,1,2,1,1,2,1,1,2,2,2,2,2,3,2,3,3,2,2,3,1,1,...",000000003
3,"0,1,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,...",000000009
4,"3,8,4,5,7,1,7,5,0,5,0,4,9,4,8,7,0,3,1,4,7,9,4,...",000000007
...,...,...
3871,"1,1,1,1,2,1,1,3,3,1,1,4,6,4,1,1,6,10,10,6,1,1,...",000021259
3872,"2,7,6,8,6,7,8,2,8,2,9,8,7,3,2,1,5,3,2,4,9,5,3,...",000000006
3873,"1,1,1,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...",000000005
3874,"1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",000000002


In [6]:
data['TrainSeq']

0       1,1,9,1,25,11,1,49,31,43,1,81,61,247,85,1,121,...
1       1,0,0,1,0,0,1,0,0,1,0,0,2,0,0,2,0,0,2,0,0,2,0,...
2       1,1,1,2,1,1,2,1,1,2,2,2,2,2,3,2,3,3,2,2,3,1,1,...
3       0,1,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,...
4       3,8,4,5,7,1,7,5,0,5,0,4,9,4,8,7,0,3,1,4,7,9,4,...
                              ...                        
3871    1,1,1,1,2,1,1,3,3,1,1,4,6,4,1,1,6,10,10,6,1,1,...
3872    2,7,6,8,6,7,8,2,8,2,9,8,7,3,2,1,5,3,2,4,9,5,3,...
3873    1,1,1,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...
3874    1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...
3875    1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,...
Name: TrainSeq, Length: 3876, dtype: object

In [7]:
input_length = len(data['TrainSeq'][0])
encoded = []
symbols = '0123456789,-'

for sequence in data['TrainSeq']:
    flattened = torch.flatten(seqToTensor(sequence, input_length, symbols = symbols))
    encoded.append(flattened)

In [8]:
# Load it in an efficient way

In [19]:
train_dataset = CSVDataset('data/preprocessed-1-dnn.csv')
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Define Model

In [20]:
''' Define key attributes '''
input_size = encoded[0].shape[0]
output_size = len(symbols) * 9

In [21]:
model = DNN(input_size, output_size, symbol_set = symbols,seed = 5, hidden_layer_param=[640])

In [22]:
''' Test a Forward pass '''
output = model(encoded[0])

427661303


# Training

In [29]:
def run_training(model, train_dataloader, epochs=1):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            encoded_inputs = torch.flatten(seqToTensor(inputs[0], input_length, symbols = symbols))
            print(encoded_inputs)


            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(encoded_inputs)
            


            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

In [30]:
run_training(model, train_dataloader, epochs=1)

tensor([0., 0., 1.,  ..., 0., 0., 0.])
410657302


AttributeError: 'int' object has no attribute 'log_softmax'

# Testing it

In [ ]:
def test_run(model):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    return 100 * correct / total